In [2]:
import numpy as np
import pandas as pd
import pybithumb
import os
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from Funcs_CNN4 import obv
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)

Scaler = MinMaxScaler()

dir = 'ohlcv/'
ohlcv_list = os.listdir(dir)


def min_max_scaler(price):
    Scaler = MinMaxScaler()
    Scaler.fit(price)

    return Scaler.transform(price)


def low_high(Coin, input_data_length, ip_limit=None, trade_limit=None, crop_size=300, sudden_death=0):

    #   거래 제한은 고점과 저점을 분리한다.

    #   User-Agent Configuration
    #   IP - Change
    if ip_limit is None:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1')
    else:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1', 'proxyon')

    price_gap = ohlcv_excel.close.max() / ohlcv_excel.close.min()
    if (price_gap < 1.07) and (trade_limit is not None):
        return None, None, None

    obv = [0] * len(ohlcv_excel)
    for m in range(1, len(ohlcv_excel)):
        if ohlcv_excel['close'].iloc[m] > ohlcv_excel['close'].iloc[m - 1]:
            obv[m] = obv[m - 1] + ohlcv_excel['volume'].iloc[m]
        elif ohlcv_excel['close'].iloc[m] == ohlcv_excel['close'].iloc[m - 1]:
            obv[m] = obv[m - 1]
        else:
            obv[m] = obv[m - 1] - ohlcv_excel['volume'].iloc[m]
    ohlcv_excel['OBV'] = obv

    closeprice = ohlcv_excel['close'].iloc[-1]
    datetime_list = ohlcv_excel.index.values

    # ----------- dataX, dataY 추출하기 -----------#
    #   OBV :
    ohlcv_data = ohlcv_excel.values[1:].astype(np.float)

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        # price = ohlcv_data[:, :4]
        # volume = ohlcv_data[:, [4]]
        # OBV = ohlcv_data[:, [-1]]
        #
        # scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_OBV = min_max_scaler(OBV)
        # print(scaled_MA60.shape)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        # print(x.shape)
        # print(ohlcv_data.shape)
        # quit()

        # if (x[-1][1] > 0.3) and (trade_limit is not None):
        #     return None, None

        # print(x.shape)  # (258, 6)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        for i in range(crop_size, len(ohlcv_data) + 1):  # 마지막 데이터까지 다 긇어모은다.
            group_x = ohlcv_data[i - crop_size: i]
            scaled_price = min_max_scaler(group_x[:, :4])
            # scaled_volume = min_max_scaler(group_x[:, [4]])
            # scaled_OBV = min_max_scaler(group_x[:, [-1]])
            # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1) + sudden_death  # axis=1, 세로로 합친다
            x = scaled_price + sudden_death  # axis=1, 세로로 합친다
            group_x = x[-input_data_length:]
            # print(group_x[-1:, 1:2])
            # print(group_x.shape)
            # print(group_x.max())
            dataX.append(group_x)  # dataX 리스트에 추가

        if len(dataX) < 100:
            return None, None, None, None

        # quit()
        X_test = np.array(dataX)
        row = X_test.shape[1]
        col = X_test.shape[2]

        X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # closeprice = np.roll(np.array(list(map(lambda x: x[-1][[1]][0], X_test))), -1)
        # plt.plot(closeprice)
        # plt.show()

        return X_test, closeprice, min_max_scaler(ohlcv_data[crop_size:, [1]]), datetime_list


def low_high_origin(Coin, input_data_length, ip_limit=None, trade_limit=None, crop_size=300, sudden_death=0):

    #   거래 제한은 고점과 저점을 분리한다.

    #   User-Agent Configuration
    #   IP - Change
    if ip_limit is None:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1')
    else:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1', 'proxyon')

    price_gap = ohlcv_excel.close.max() / ohlcv_excel.close.min()
    if (price_gap < 1.07) and (trade_limit is not None):
        return None, None, None

    obv = [0] * len(ohlcv_excel)
    for m in range(1, len(ohlcv_excel)):
        if ohlcv_excel['close'].iloc[m] > ohlcv_excel['close'].iloc[m - 1]:
            obv[m] = obv[m - 1] + ohlcv_excel['volume'].iloc[m]
        elif ohlcv_excel['close'].iloc[m] == ohlcv_excel['close'].iloc[m - 1]:
            obv[m] = obv[m - 1]
        else:
            obv[m] = obv[m - 1] - ohlcv_excel['volume'].iloc[m]
    ohlcv_excel['OBV'] = obv

    closeprice = ohlcv_excel['close'].iloc[-1]
    datetime_list = ohlcv_excel.index.values

    # ----------- dataX, dataY 추출하기 -----------#
    #   OBV :
    ohlcv_data = ohlcv_excel.values[1:].astype(np.float)

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        price = ohlcv_data[:, :4]
        volume = ohlcv_data[:, [4]]
        OBV = ohlcv_data[:, [-1]]

        scaled_price = min_max_scaler(price)
        scaled_volume = min_max_scaler(volume)
        scaled_OBV = min_max_scaler(OBV)
        # print(scaled_MA60.shape)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        x = scaled_price
        # print(x.shape)
        # print(ohlcv_data.shape)
        # quit()

        # if (x[-1][1] > 0.3) and (trade_limit is not None):
        #     return None, None

        # print(x.shape)  # (258, 6)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        for i in range(input_data_length, len(x) + 1):  # 마지막 데이터까지 다 긇어모은다.
            group_x = x[i - input_data_length:i]
            dataX.append(group_x)  # dataX 리스트에 추가

        if len(dataX) < 100:
            return None, None, None

        # quit()
        X_test = np.array(dataX)
        row = X_test.shape[1]
        col = X_test.shape[2]

        X_test = X_test.astype('float32').reshape(-1, row, col, 1)

        return X_test, closeprice, min_max_scaler(ohlcv_data[input_data_length:, [1]]), datetime_list


def made_x(file, input_data_length, model_num, check_span, get_fig, crop_size=500, sudden_death=0):

    try:
        ohlcv_excel = pd.read_excel(dir + file, index_col=0)
    except Exception as e:
        return None, None, None

    # ohlcv_excel['CMO'] = cmo(ohlcv_excel)
    # ohlcv_excel['OBV'] = obv(ohlcv_excel)
    # ohlcv_excel['RSI'] = rsi(ohlcv_excel)

#     print(ohlcv_excel.info())
#     quit()

    #   이후 check_span 데이터와 현재 포인트를 비교해서 현재 포인트가 저가인지 고가인지 예측한다.
    #   진입, 저점, 고점, 거래 안함의 y_label 인 trade_state  >> [1, 2, 0]
    #   저점과 고점은 최대 3개의 중복 값을 허용한다.
    trade_state = [np.NaN] * len(ohlcv_excel)
    for i in range(len(ohlcv_excel) - check_span):
        #   저점
        if ohlcv_excel['close'][i - check_span:i].min() >= ohlcv_excel['close'][i]:
            if ohlcv_excel['close'][i + 1:i + 1 + check_span].min() >= ohlcv_excel['close'][i]:
                # if ohlcv_excel['close'][i - check_span:i + 1 + check_span].value_counts().sort_index().iloc[0] <= 3:
                if 1 not in trade_state[i - check_span:i]:
                    trade_state[i] = 1
                else:
                    trade_state[i] = 0
                # else:
                #     trade_state[i] = 0
            else:
                trade_state[i] = 0
        #   고점
        elif ohlcv_excel['close'][i - check_span:i].max() <= ohlcv_excel['close'][i]:
            if ohlcv_excel['close'][i + 1:i + 1 + check_span].max() <= ohlcv_excel['close'][i]:
                # if ohlcv_excel['close'][i - check_span:i + 1 + check_span].value_counts().sort_index().iloc[-1] <= 3:
                if 2 not in trade_state[i - check_span:i]:
                    trade_state[i] = 2
                else:
                    trade_state[i] = 0
            # else:
            #     trade_state[i] = 0
            else:
                trade_state[i] = 0

        #   거래 안함
        else:
            trade_state[i] = 0

    ohlcv_excel['trade_state'] = trade_state

    # ----------- dataX, dataY 추출하기 -----------#
    # print(ohlcv_excel.info())
    # ohlcv_excel.to_excel('test.xlsx')
    # quit()

    # NaN 제외하고 데이터 자르기 (데이터가 PIXEL 로 들어간다고 생각하면 된다)
    #   OBV : -CHECK_SPAN
    ohlcv_data = ohlcv_excel.values[: -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[1: -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.CMO.isna()): -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.RSI.isna()): -check_span].astype(np.float)

    # print(pd.DataFrame(ohlcv_data).info())
    # print(pd.DataFrame(ohlcv_data).to_excel('test.xlsx'))
    # print(list(map(float, ohlcv_data[0])))
    # quit()

    #       출력되는 Y 데이터 길이를 동일화 시킨다.     #
    # ohlcv_data = ohlcv_data[-300:]

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        price = ohlcv_data[:, :4]
        # volume = ohlcv_data[:, [4]]
        # CMO = ohlcv_data[:, [-2]]
        # OBV = ohlcv_data[:, [-2]]
        # RSI = ohlcv_data[:, [-2]]

        #   Flexible Y_data    #
        trade_state = ohlcv_data[:, [-1]]

        # scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_CMO = min_max_scaler(CMO)
        # scaled_OBV = min_max_scaler(OBV)
        # scaled_RSI = min_max_scaler(RSI)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        # x = scaled_price
        y = trade_state
        # print(x.shape, y_low.shape)  # (258, 6) (258, 1)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        dataY = []  # Target 을 담을 그릇
        for i in range(crop_size, len(ohlcv_data)):  # 마지막 데이터까지 다 긇어모은다.
            group_x = ohlcv_data[i - crop_size: i]
            group_y = y[i]
            scaled_price = min_max_scaler(group_x[:, :4])
            x = scaled_price + sudden_death  # axis=1, 세로로 합친다
            group_x = x[-input_data_length:]

            dataX.append(group_x)  # dataX 리스트에 추가
            dataY.append(group_y)

        if len(dataX) < 100:
            print('len(dataX) < 100')
            return None, None, None

        X_test = np.array(dataX)
        row = X_test.shape[1]
        col = X_test.shape[2]

        X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # print(X_test.shape)

        #       Exstracting fiexd X_data       #
        sliced_ohlcv = ohlcv_data[crop_size:, :x.shape[1]]

        #                      Get Figure                     #
        if get_fig == 1:
            spanlist_low = []
            spanlist_high = []

            for m in range(len(trade_state)):
                if (trade_state[m] > 0.5) and (trade_state[m] < 1.5):
                    if m + 1 < len(trade_state):
                        spanlist_low.append((m, m + 1))
                    else:
                        spanlist_low.append((m - 1, m))

            for m in range(len(trade_state)):
                if (trade_state[m] > 1.5) and (trade_state[m] < 2.5):
                    if m + 1 < len(trade_state):
                        spanlist_high.append((m, m + 1))
                    else:
                        spanlist_high.append((m - 1, m))

            # ----------- 인덱스 초기화 됨 -----------#

            # ----------- 공통된 Chart 그리기 -----------#

            plt.subplot(211)
            plt.plot(min_max_scaler(ohlcv_data[:, 1:2]), 'r', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_low)):
                plt.axvspan(spanlist_low[i][0], spanlist_low[i][1], facecolor='m', alpha=0.5)

            plt.subplot(212)
            plt.plot(min_max_scaler(ohlcv_data[:, 1:2]), 'r', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_high)):
                plt.axvspan(spanlist_high[i][0], spanlist_high[i][1], facecolor='c', alpha=0.5)

            Date = file.split()[0]
            Coin = file.split()[1].split('.')[0]
            plt.savefig('./Figure_data/%s_%s/%s %s.png' % (input_data_length, model_num, Date, Coin), dpi=500)
            plt.close()
            # plt.show()
            # ----------- Chart 그리기 -----------#

        return dataX, dataY, sliced_ohlcv


def made_x_origin(file, input_data_length, model_num, check_span, get_fig, crop_size=0, sudden_death=0):

    if type(file) is str:
        ohlcv_excel = pd.read_excel(dir + file, index_col=0)

        # ohlcv_excel['MA60'] = ohlcv_excel['close'].rolling(60).mean()
        # ohlcv_excel['CMO'] = cmo(ohlcv_excel)
        # ohlcv_excel['OBV'] = obv(ohlcv_excel)
        # ohlcv_excel['RSI'] = rsi(ohlcv_excel)

        # print(ohlcv_excel)
        # quit()

        #   이후 check_span 데이터와 현재 포인트를 비교해서 현재 포인트가 저가인지 고가인지 예측한다.
        #   진입, 저점, 고점, 거래 안함의 y_label 인 trade_state  >> [1, 2, 0]
        #   저점과 고점은 최대 3개의 중복 값을 허용한다.
        trade_state = [np.NaN] * len(ohlcv_excel)
        for i in range(len(ohlcv_excel) - check_span):
            #   저점
            if ohlcv_excel['close'][i + 1:i + 1 + check_span].min() >= ohlcv_excel['close'][i]:
                if ohlcv_excel['close'][i:i + 1 + check_span].value_counts().sort_index().iloc[0] <= 3:
                    trade_state[i] = 1
                else:
                    trade_state[i] = 0
            #   고점
            elif ohlcv_excel['close'][i + 1:i + 1 + check_span].max() <= ohlcv_excel['close'][i]:
                if ohlcv_excel['close'][i:i + 1 + check_span].value_counts().sort_index().iloc[-1] <= 3:
                    trade_state[i] = 2
                else:
                    trade_state[i] = 0
            #   거래 안함
            else:
                trade_state[i] = 0

        ohlcv_excel['trade_state'] = trade_state

        # ----------- dataX, dataY 추출하기 -----------#
        # print(ohlcv_excel.info())
        # ohlcv_excel.to_excel('test.xlsx')
        # quit()

        # NaN 제외하고 데이터 자르기 (데이터가 PIXEL 로 들어간다고 생각하면 된다)
        #   OBV : -CHECK_SPAN
        ohlcv_data = ohlcv_excel.values[: -check_span].astype(np.float)
        # ohlcv_data = ohlcv_excel.values[ohlcv_excel['MA60'].isnull().sum():-check_span - 150].astype(np.float)
        # ohlcv_data = ohlcv_excel.values[1: -check_span].astype(np.float)
        # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.CMO.isna()): -check_span].astype(np.float)
        # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.RSI.isna()): -check_span].astype(np.float)

        # print(pd.DataFrame(ohlcv_data).info())
        # print(pd.DataFrame(ohlcv_data).to_excel('test.xlsx'))
        # print(list(map(float, ohlcv_data[0])))
        # quit()
    else:
        ohlcv_data = file

    #           ohlcv_data slicing      #
    # ohlcv_data = ohlcv_data[-crop_size:]

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        price = ohlcv_data[:, :4]
        # volume = ohlcv_data[:, [4]]
        # MA60 = ohlcv_data[:, [-3]]
        # CMO = ohlcv_data[:, [-2]]
        # OBV = ohlcv_data[:, [-2]]
        # RSI = ohlcv_data[:, [-2]]

        #   Flexible Y_data    #
        trade_state = ohlcv_data[:, [-1]]

        scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_MA60 = min_max_scaler(MA60)
        # scaled_CMO = min_max_scaler(CMO)
        # scaled_OBV = min_max_scaler(OBV)
        # scaled_RSI = min_max_scaler(RSI)

        # x = np.concatenate((scaled_price, scaled_MA60), axis=1)  # axis=1, 세로로 합친다
        x = scaled_price
        y = trade_state
        # print(x.shape, y_low.shape)  # (258, 6) (258, 1)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        dataY = []  # Target 을 담을 그릇
        for i in range(input_data_length, len(x)):
            group_x = x[i - input_data_length:i]
            group_y = y[i]

            dataX.append(group_x)  # dataX 리스트에 추가
            dataY.append(group_y)

        if len(dataX) < 100:
            return None, None, None

        X_test = np.array(dataX)
        row = X_test.shape[1]
        col = X_test.shape[2]

        X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # print(X_test.shape)

        #       Exstracting fiexd X_data       #
        sliced_ohlcv = ohlcv_data[input_data_length:, :x.shape[1]]

        #                      Get Figure                     #
        if get_fig == 1:
            spanlist_low = []
            spanlist_high = []

            for m in range(len(trade_state)):
                if (trade_state[m] > 0.5) and (trade_state[m] < 1.5):
                    if m + 1 < len(trade_state):
                        spanlist_low.append((m, m + 1))
                    else:
                        spanlist_low.append((m - 1, m))

            for m in range(len(trade_state)):
                if (trade_state[m] > 1.5) and (trade_state[m] < 2.5):
                    if m + 1 < len(trade_state):
                        spanlist_high.append((m, m + 1))
                    else:
                        spanlist_high.append((m - 1, m))

            # ----------- 인덱스 초기화 됨 -----------#

            # ----------- 공통된 Chart 그리기 -----------#

            plt.subplot(211)
            plt.plot(min_max_scaler(ohlcv_data[:, 1:2]), 'r', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_low)):
                plt.axvspan(spanlist_low[i][0], spanlist_low[i][1], facecolor='m', alpha=0.5)

            plt.subplot(212)
            plt.plot(min_max_scaler(ohlcv_data[:, 1:2]), 'r', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_high)):
                plt.axvspan(spanlist_high[i][0], spanlist_high[i][1], facecolor='c', alpha=0.5)

            Date = file.split()[0]
            Coin = file.split()[1].split('.')[0]
            plt.savefig('./Figure_data/%s_%s/%s %s.png' % (input_data_length, model_num, Date, Coin), dpi=500)
            plt.close()
            # plt.show()
        
        return X_test, dataY, sliced_ohlcv


if __name__ == '__main__':

    # ----------- Params -----------#
    input_data_length = 50
    model_num = 50

    #       Make folder      #
    try:
        os.mkdir('./Figure_data/%s_%s/' % (input_data_length, model_num))

    except Exception as e:
        pass
    check_span = 30
    get_fig = 0

    Made_X = []
    Made_Y = []

#     ohlcv_list = ['2020-01-10 ETH ohlcv.xlsx']

    for file in ohlcv_list:
        try:
            if int(file.split()[0].split('-')[1]) != 1:
                continue


            result = made_x(file, input_data_length, model_num, check_span, get_fig, crop_size=input_data_length)
            # result = low_high('dvp'.upper(), input_data_length)
            # print(result)
            # quit()

            # ------------ 데이터가 있으면 dataX, dataY 병합하기 ------------#
            if result is not None:
                if result[0] is not None:

                    Made_X += result[0]
                    Made_Y += result[1]

                # 누적 데이터량 표시
                print(file, len(Made_X))
                
        except Exception as e:
            pass

    # SAVING X, Y
    X = np.array(Made_X)
    Y = np.array(Made_Y)
    np.save('./Made_X/Made_X %s_%s' % (input_data_length, model_num), X)
    np.save('./Made_X/Made_Y %s_%s' % (input_data_length, model_num), Y)
    # np.save('./Made_X_high/Made_Y %s_%s' % (input_data_length, model_num), Y_high)



2020-01-16 XLM ohlcv.xlsx 1010
2020-01-23 TRX ohlcv.xlsx 1740
2020-01-31 AMO ohlcv.xlsx 2397
2020-01-16 SNT ohlcv.xlsx 2619
len(dataX) < 100
2020-01-19 LBA ohlcv.xlsx 2619
2020-01-11 FX ohlcv.xlsx 2942
2020-01-22 LUNA ohlcv.xlsx 4271
2020-01-16 LINK ohlcv.xlsx 5272
len(dataX) < 100
2020-01-28 PLY ohlcv.xlsx 5272
2020-01-13 FCT ohlcv.xlsx 5484
2020-01-20 IPX ohlcv.xlsx 6843
len(dataX) < 100
2020-01-27 ITC ohlcv.xlsx 6843
len(dataX) < 100
2020-01-14 FZZ ohlcv.xlsx 6843
len(dataX) < 100
2020-01-24 BZNT ohlcv.xlsx 6843
2020-01-20 SNT ohlcv.xlsx 7100
2020-01-29 TMTG ohlcv.xlsx 8019
len(dataX) < 100
2020-01-25 VET ohlcv.xlsx 8019
2020-01-21 EOS ohlcv.xlsx 9186
2020-01-24 FAB ohlcv.xlsx 9358
2020-01-22 TRV ohlcv.xlsx 9554
len(dataX) < 100
2020-01-13 MBL ohlcv.xlsx 9554
len(dataX) < 100
2020-01-26 DVP ohlcv.xlsx 9554
2020-01-22 MTL ohlcv.xlsx 9936
len(dataX) < 100
2020-01-12 POWR ohlcv.xlsx 9936
2020-01-28 TRX ohlcv.xlsx 10617
2020-01-31 LAMB ohlcv.xlsx 10733
len(dataX) < 100
2020-01-12 MCO oh

2020-01-28 AOA ohlcv.xlsx 72527
len(dataX) < 100
2020-01-14 XEM ohlcv.xlsx 72527
2020-01-20 WAVES ohlcv.xlsx 72679
len(dataX) < 100
2020-01-11 CHR ohlcv.xlsx 72679
2020-01-28 BCD ohlcv.xlsx 73506
len(dataX) < 100
2020-01-30 PIVX ohlcv.xlsx 73506
2020-01-22 CON ohlcv.xlsx 74554
len(dataX) < 100
2020-01-29 XMR ohlcv.xlsx 74554
len(dataX) < 100
2020-01-28 POWR ohlcv.xlsx 74554
2020-01-25 MTL ohlcv.xlsx 74832
2020-01-24 ABT ohlcv.xlsx 74950
2020-01-24 ETC ohlcv.xlsx 75746
2020-01-28 REP ohlcv.xlsx 76178
len(dataX) < 100
2020-01-14 WAVES ohlcv.xlsx 76178
len(dataX) < 100
2020-01-22 ANKR ohlcv.xlsx 76178
2020-01-20 AE ohlcv.xlsx 76559
len(dataX) < 100
2020-01-19 TRUE ohlcv.xlsx 76559
len(dataX) < 100
2020-01-28 VET ohlcv.xlsx 76559
len(dataX) < 100
2020-01-22 ABT ohlcv.xlsx 76559
2020-01-30 BTC ohlcv.xlsx 77904
2020-01-31 XMR ohlcv.xlsx 78004
2020-01-13 DAD ohlcv.xlsx 78457
len(dataX) < 100
2020-01-25 OMG ohlcv.xlsx 78457
2020-01-20 MTL ohlcv.xlsx 79237
len(dataX) < 100
2020-01-15 FZZ ohlcv.

2020-01-19 AMO ohlcv.xlsx 146915
2020-01-29 ENJ ohlcv.xlsx 147137
len(dataX) < 100
2020-01-22 LRC ohlcv.xlsx 147137
2020-01-21 DAC ohlcv.xlsx 148333
len(dataX) < 100
2020-01-24 WICC ohlcv.xlsx 148333
len(dataX) < 100
2020-01-25 ELF ohlcv.xlsx 148333
len(dataX) < 100
2020-01-15 POWR ohlcv.xlsx 148333
2020-01-15 BTT ohlcv.xlsx 148759
len(dataX) < 100
2020-01-28 CMT ohlcv.xlsx 148759
len(dataX) < 100
2020-01-29 OGO ohlcv.xlsx 148759
2020-01-30 ZIL ohlcv.xlsx 148917
2020-01-16 ARN ohlcv.xlsx 149135
2020-01-19 DAD ohlcv.xlsx 149261
len(dataX) < 100
2020-01-15 VALOR ohlcv.xlsx 149261
len(dataX) < 100
2020-01-23 WICC ohlcv.xlsx 149261
len(dataX) < 100
2020-01-25 ORBS ohlcv.xlsx 149261
2020-01-24 XSR ohlcv.xlsx 150260
2020-01-15 LAMB ohlcv.xlsx 150421
2020-01-29 LINK ohlcv.xlsx 150879
len(dataX) < 100
2020-01-30 GXC ohlcv.xlsx 150879
len(dataX) < 100
2020-01-15 MBL ohlcv.xlsx 150879
2020-01-16 REP ohlcv.xlsx 151645
len(dataX) < 100
2020-01-14 THETA ohlcv.xlsx 151645
2020-01-22 TRX ohlcv.xlsx 1

2020-01-31 RDN ohlcv.xlsx 207916
2020-01-22 FAB ohlcv.xlsx 208394
len(dataX) < 100
2020-01-15 RDN ohlcv.xlsx 208394
2020-01-20 OMG ohlcv.xlsx 208575
2020-01-30 VET ohlcv.xlsx 208726
len(dataX) < 100
2020-01-24 RNT ohlcv.xlsx 208726
2020-01-30 XLM ohlcv.xlsx 209546
len(dataX) < 100
2020-01-29 CHR ohlcv.xlsx 209546
2020-01-12 REP ohlcv.xlsx 209860
len(dataX) < 100
2020-01-28 ANKR ohlcv.xlsx 209860
2020-01-15 OGO ohlcv.xlsx 210199
2020-01-14 LAMB ohlcv.xlsx 210317
2020-01-13 MIX ohlcv.xlsx 210485
2020-01-28 BSV ohlcv.xlsx 211743
2020-01-15 WAVES ohlcv.xlsx 211883
len(dataX) < 100
2020-01-13 ORBS ohlcv.xlsx 211883
2020-01-25 DAC ohlcv.xlsx 213239
2020-01-12 HC ohlcv.xlsx 213351
2020-01-13 ADA ohlcv.xlsx 213623
2020-01-16 LAMB ohlcv.xlsx 214082
len(dataX) < 100
2020-01-26 XMR ohlcv.xlsx 214082
2020-01-12 FCT ohlcv.xlsx 214507
len(dataX) < 100
2020-01-31 MXC ohlcv.xlsx 214507
len(dataX) < 100
2020-01-14 BAT ohlcv.xlsx 214507
2020-01-29 ICX ohlcv.xlsx 214771
len(dataX) < 100
2020-01-23 KNC oh

2020-01-30 ETH ohlcv.xlsx 273099
len(dataX) < 100
2020-01-19 WET ohlcv.xlsx 273099
2020-01-19 REP ohlcv.xlsx 273856
len(dataX) < 100
2020-01-23 CTXC ohlcv.xlsx 273856
len(dataX) < 100
2020-01-20 BAT ohlcv.xlsx 273856
2020-01-19 BTG ohlcv.xlsx 274950
2020-01-20 ZEC ohlcv.xlsx 275424
len(dataX) < 100
2020-01-19 BHP ohlcv.xlsx 275424
len(dataX) < 100
2020-01-28 OMG ohlcv.xlsx 275424
2020-01-30 IOST ohlcv.xlsx 275555
2020-01-22 IOST ohlcv.xlsx 275675
2020-01-23 PIVX ohlcv.xlsx 275809
2020-01-30 BTT ohlcv.xlsx 276173
len(dataX) < 100
2020-01-21 XVG ohlcv.xlsx 276173
2020-01-12 BHP ohlcv.xlsx 277151
2020-01-23 WPX ohlcv.xlsx 278079
2020-01-26 BCD ohlcv.xlsx 278333
len(dataX) < 100
2020-01-29 KNC ohlcv.xlsx 278333
2020-01-15 SOC ohlcv.xlsx 279650
len(dataX) < 100
2020-01-12 THETA ohlcv.xlsx 279650
2020-01-21 BTC ohlcv.xlsx 280999
len(dataX) < 100
2020-01-23 BHP ohlcv.xlsx 280999
len(dataX) < 100
2020-01-29 STRAT ohlcv.xlsx 280999
len(dataX) < 100
2020-01-22 PLY ohlcv.xlsx 280999
len(dataX) < 

2020-01-30 APIS ohlcv.xlsx 339283
2020-01-12 LUNA ohlcv.xlsx 340489
len(dataX) < 100
2020-01-22 ORBS ohlcv.xlsx 340489
2020-01-11 OGO ohlcv.xlsx 341322
len(dataX) < 100
2020-01-11 ANKR ohlcv.xlsx 341322
len(dataX) < 100
2020-01-15 GXC ohlcv.xlsx 341322
2020-01-20 WPX ohlcv.xlsx 342040
2020-01-30 MCO ohlcv.xlsx 342215
2020-01-29 REP ohlcv.xlsx 342592
2020-01-15 ZIL ohlcv.xlsx 342692
len(dataX) < 100
2020-01-21 QKC ohlcv.xlsx 342692
len(dataX) < 100
2020-01-25 ZEC ohlcv.xlsx 342692
2020-01-20 FCT ohlcv.xlsx 342978
len(dataX) < 100
2020-01-24 ZIL ohlcv.xlsx 342978
2020-01-25 ETH ohlcv.xlsx 344075
len(dataX) < 100
2020-01-21 ICX ohlcv.xlsx 344075
len(dataX) < 100
2020-01-27 WAXP ohlcv.xlsx 344075
len(dataX) < 100
2020-01-20 THETA ohlcv.xlsx 344075
2020-01-15 MIX ohlcv.xlsx 344218
2020-01-14 FNB ohlcv.xlsx 344758
len(dataX) < 100
2020-01-15 WET ohlcv.xlsx 344758
2020-01-27 ETC ohlcv.xlsx 345338
2020-01-21 APIS ohlcv.xlsx 345505
2020-01-27 LINK ohlcv.xlsx 345739
len(dataX) < 100
2020-01-22 E

2020-01-24 BSV ohlcv.xlsx 401935
2020-01-30 INS ohlcv.xlsx 402143
len(dataX) < 100
2020-01-30 LOOM ohlcv.xlsx 402143
2020-01-20 INS ohlcv.xlsx 402686
2020-01-20 XMR ohlcv.xlsx 402796
len(dataX) < 100
2020-01-26 ORBS ohlcv.xlsx 402796
len(dataX) < 100
2020-01-28 SNT ohlcv.xlsx 402796
len(dataX) < 100
2020-01-28 PIVX ohlcv.xlsx 402796
2020-01-19 WAVES ohlcv.xlsx 403020
2020-01-27 BCH ohlcv.xlsx 403916
2020-01-29 LUNA ohlcv.xlsx 404889
2020-01-27 IPX ohlcv.xlsx 406164
len(dataX) < 100
2020-01-30 LBA ohlcv.xlsx 406164
2020-01-29 SOC ohlcv.xlsx 406933
len(dataX) < 100
2020-01-12 SNT ohlcv.xlsx 406933
len(dataX) < 100
2020-01-19 ORBS ohlcv.xlsx 406933
2020-01-26 XLM ohlcv.xlsx 407194
2020-01-11 PLY ohlcv.xlsx 407616
len(dataX) < 100
2020-01-15 LRC ohlcv.xlsx 407616
len(dataX) < 100
2020-01-26 VET ohlcv.xlsx 407616
2020-01-14 CON ohlcv.xlsx 408494
len(dataX) < 100
2020-01-24 NPXS ohlcv.xlsx 408494
len(dataX) < 100
2020-01-31 PPT ohlcv.xlsx 408494
2020-01-23 QTUM ohlcv.xlsx 409128
2020-01-16 D

len(dataX) < 100
2020-01-20 MCO ohlcv.xlsx 470374
2020-01-23 FAB ohlcv.xlsx 470687
len(dataX) < 100
2020-01-25 WET ohlcv.xlsx 470687
len(dataX) < 100
2020-01-31 WAXP ohlcv.xlsx 470687
len(dataX) < 100
2020-01-27 WICC ohlcv.xlsx 470687
2020-01-11 ETZ ohlcv.xlsx 471303
2020-01-29 CON ohlcv.xlsx 472255
len(dataX) < 100
2020-01-12 HDAC ohlcv.xlsx 472255
2020-01-30 EOS ohlcv.xlsx 473445
len(dataX) < 100
2020-01-27 INS ohlcv.xlsx 473445
2020-01-31 XLM ohlcv.xlsx 474098
len(dataX) < 100
2020-01-30 LAMB ohlcv.xlsx 474098
2020-01-23 ABT ohlcv.xlsx 474337
len(dataX) < 100
2020-01-22 BHP ohlcv.xlsx 474337
len(dataX) < 100
2020-01-19 RNT ohlcv.xlsx 474337
2020-01-24 EOS ohlcv.xlsx 475275
2020-01-21 ETC ohlcv.xlsx 476367
len(dataX) < 100
2020-01-29 CTXC ohlcv.xlsx 476367
2020-01-16 DASH ohlcv.xlsx 477223
2020-01-12 TMTG ohlcv.xlsx 478513
2020-01-26 BCH ohlcv.xlsx 479408
len(dataX) < 100
2020-01-15 WICC ohlcv.xlsx 479408
2020-01-31 CHR ohlcv.xlsx 480532
len(dataX) < 100
2020-01-31 CTXC ohlcv.xlsx 48

2020-01-19 HDAC ohlcv.xlsx 533327
2020-01-20 ENJ ohlcv.xlsx 533534
2020-01-29 QTUM ohlcv.xlsx 534284
len(dataX) < 100
2020-01-30 ITC ohlcv.xlsx 534284
2020-01-25 PLY ohlcv.xlsx 534511
2020-01-21 ADA ohlcv.xlsx 535060
2020-01-15 ETZ ohlcv.xlsx 535270
len(dataX) < 100
2020-01-15 FAB ohlcv.xlsx 535270
2020-01-30 SOC ohlcv.xlsx 536209
2020-01-13 BHP ohlcv.xlsx 537146
2020-01-22 ZIL ohlcv.xlsx 537272
len(dataX) < 100
2020-01-28 PAY ohlcv.xlsx 537272
len(dataX) < 100
2020-01-25 XVG ohlcv.xlsx 537272
2020-01-11 BTC ohlcv.xlsx 538625
len(dataX) < 100
2020-01-15 CHR ohlcv.xlsx 538625
2020-01-31 REP ohlcv.xlsx 538966
2020-01-29 FCT ohlcv.xlsx 539339
2020-01-28 FCT ohlcv.xlsx 539588
2020-01-29 WPX ohlcv.xlsx 540596
2020-01-31 XEM ohlcv.xlsx 540699
len(dataX) < 100
2020-01-12 STRAT ohlcv.xlsx 540699
2020-01-14 BCD ohlcv.xlsx 541127
len(dataX) < 100
2020-01-19 QKC ohlcv.xlsx 541127
2020-01-26 MTL ohlcv.xlsx 541356
2020-01-13 ABT ohlcv.xlsx 541702
len(dataX) < 100
2020-01-23 BZNT ohlcv.xlsx 541702
l

2020-01-31 DASH ohlcv.xlsx 591433
len(dataX) < 100
2020-01-27 RDN ohlcv.xlsx 591433
len(dataX) < 100
2020-01-21 DVP ohlcv.xlsx 591433
len(dataX) < 100
2020-01-26 DASH ohlcv.xlsx 591433
len(dataX) < 100
2020-01-13 XMR ohlcv.xlsx 591433
len(dataX) < 100
2020-01-13 IOST ohlcv.xlsx 591433
2020-01-26 ETC ohlcv.xlsx 592070
len(dataX) < 100
2020-01-30 KNC ohlcv.xlsx 592070
len(dataX) < 100
2020-01-22 LAMB ohlcv.xlsx 592070
len(dataX) < 100
2020-01-15 HDAC ohlcv.xlsx 592070
len(dataX) < 100
2020-01-13 CMT ohlcv.xlsx 592070
len(dataX) < 100
2020-01-19 ITC ohlcv.xlsx 592070
2020-01-23 LTC ohlcv.xlsx 592535
len(dataX) < 100
2020-01-13 MXC ohlcv.xlsx 592535
2020-01-29 BCD ohlcv.xlsx 593395
len(dataX) < 100
2020-01-15 QKC ohlcv.xlsx 593395
2020-01-31 BCD ohlcv.xlsx 593933
2020-01-29 ETH ohlcv.xlsx 595161
2020-01-21 THETA ohlcv.xlsx 595285
2020-01-19 WPX ohlcv.xlsx 596229
2020-01-27 GXC ohlcv.xlsx 596332
len(dataX) < 100
2020-01-25 CTXC ohlcv.xlsx 596332
len(dataX) < 100
2020-01-29 GNT ohlcv.xlsx 59